# 2014222 - Semester 2 CA-02 -May 2024

### Github account

https://github.com/2014222-student-cct-ie/2024--Semester-2--CA2/

# Part 2

In [1]:
# Utilise Python programming language in order to comply with the requisites of the assessment and perform adequate Machine
# Learning algorithms to discover and deliver insights.

# Import the necessary libraries (Numpy and Pandas) in order to perform data cleansing.
# These are the libraries that are conventionally used as a common practice in order to
# perform mathematical and statistical operations during a data analysis process

import numpy as np
import pandas as pd

# Import Matplotlib and Plotly library in order to perform data visualisation procedures

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

# Import Seaborn libraries in order to perform data visualisation procedures

import seaborn as sns

# I am using this line of code to see all columns in a wide DataFrame

pd.set_option('display.max_columns', None)

from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col
from pyspark.ml import Pipeline
from pyspark.sql.types import FloatType


# For normalization
from pyspark.ml.feature import MinMaxScaler 
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit


# process the tweets data
# !pip install textblob
from pyspark.sql.functions import udf
from textblob import TextBlob

# pip install nltk
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

from pyspark.sql import DataFrame

# re module provides regular expression support.
# In Python a regular expression search is typically written as:
# match = re. search(pat, str)
# The re.search() method takes a regular expression pattern and a string and searches
# for that pattern within the string.

import re

from pyspark.sql.types import StructType, StructField, IntegerType, StringType

#!pip install pyspark

#!pip install pyspark pandas

# Import the warnings module

import warnings

# Ignore all warnings by applying th the 'filterwarnings()'' function and passing the 'ignore' argument

warnings.filterwarnings('ignore')

/Users/geomarmunoz/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Note

I did the sentiment analysis usign Pyspark. However, when I tried to export the results to a CSV it crashes after.

Therefore I will do the sentiment analysis again **this time using pandas**, but using this time cleaned_tweet_dataset.csv where all the tweets are cleaned (no @ mentions, hashtags, retweets, hyperlinks, and colons, emojis, unicode characters from a string, emoticons, dingbats, symbols & pictographs, transport & map symbols, flags (iOS), Chinese characters, etc.)

The error is captured in an image in part 1

In [2]:
tweets_dataset = pd.read_csv('cleaned_tweet_dataset.csv')

In [3]:
tweets_dataset

,tweet_ID,date_timestamp,twitter_user,cleaned_tweet_text
0,1467810369,04-06-2009 22:19:45,_TheSpecialOne_,"- Awww, that's a bummer. You shoulda got Davi..."
1,1467810672,04-06-2009 22:19:49,scotthamilton,is upset that he can't update his Facebook by ...
2,1467810917,04-06-2009 22:19:53,mattycus,I dived many times for the ball. Managed to sa...
3,1467811184,04-06-2009 22:19:57,ElleCTF,my whole body feels itchy and like its on fire
4,1467811193,04-06-2009 22:19:57,Karoli,"no, it's not behaving at all. i'm mad. why am ..."
...,...,...,...,...
638474,2072531933,06-07-2009 21:12:35,Neo_drone,Apparetly they just started filming so imma go...
638475,2072531954,06-07-2009 21:12:35,edenhig,I would just like to say thank you from all of...
638476,2072532046,06-07-2009 21:12:36,Gotjuicygirl,thanx love ! I definitely agree
638477,2072532066,06-07-2009 21:12:36,whitet,"the ones from yesterday w/ the animals, VERY CUTE"


In [4]:
tweets_dataset.head(10)

,tweet_ID,date_timestamp,twitter_user,cleaned_tweet_text
0,1467810369,04-06-2009 22:19:45,_TheSpecialOne_,"- Awww, that's a bummer. You shoulda got Davi..."
1,1467810672,04-06-2009 22:19:49,scotthamilton,is upset that he can't update his Facebook by ...
2,1467810917,04-06-2009 22:19:53,mattycus,I dived many times for the ball. Managed to sa...
3,1467811184,04-06-2009 22:19:57,ElleCTF,my whole body feels itchy and like its on fire
4,1467811193,04-06-2009 22:19:57,Karoli,"no, it's not behaving at all. i'm mad. why am ..."
5,1467811372,04-06-2009 22:20:00,joy_wolf,not the whole crew
6,1467811592,04-06-2009 22:20:03,mybirch,Need a hug
7,1467811594,04-06-2009 22:20:03,coZZ,"hey long time no see! Yes.. Rains a bit ,only..."
8,1467811795,04-06-2009 22:20:05,2Hood4Hollywood,nope they didn't have it
9,1467812025,04-06-2009 22:20:09,mimismo,que me muera ?


In [5]:
tweets_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638479 entries, 0 to 638478
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   tweet_ID            638479 non-null  int64 
 1   date_timestamp      638479 non-null  object
 2   twitter_user        638479 non-null  object
 3   cleaned_tweet_text  637286 non-null  object
dtypes: int64(1), object(3)
memory usage: 19.5+ MB


In [6]:
# tweets_dataset.to_csv('test.csv')

# tweets_dataset_new = pd.read_csv('test.csv')

# tweets_dataset_new.head(10)

In [7]:
# tweets_dataset_new.info()

# TextBlob Sentiment Analysis

In [8]:
# Ensure the tweet text column is string
tweets_dataset['cleaned_tweet_text'] = tweets_dataset['cleaned_tweet_text'].astype(str)

# Define the sentiment analysis function with error handling
def analyze_sentiment(text):
    if isinstance(text, str) and text.strip():
        analysis = TextBlob(text)
        return analysis.sentiment.polarity
    else:
        return None  # or 0.0 if you prefer to treat non-valid texts as neutral

In [9]:
# Apply sentiment analysis
tweets_dataset['score_TextBlob'] = tweets_dataset['cleaned_tweet_text'].apply(analyze_sentiment)

In [10]:
# Display results

tweets_dataset[['cleaned_tweet_text', 'score_TextBlob']].head(10)

,cleaned_tweet_text,score_TextBlob
0,"- Awww, that's a bummer. You shoulda got Davi...",0.216667
1,is upset that he can't update his Facebook by ...,0.000000
2,I dived many times for the ball. Managed to sa...,0.500000
3,my whole body feels itchy and like its on fire,0.200000
4,"no, it's not behaving at all. i'm mad. why am ...",-0.625000
5,not the whole crew,0.200000
6,Need a hug,0.000000
7,"hey long time no see! Yes.. Rains a bit ,only...",0.270833
8,nope they didn't have it,0.000000
9,que me muera ?,0.000000


# Vader Sentiment Analysis

In [11]:
# Ensure VADER lexicon is downloaded
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/geomarmunoz/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [12]:
# Initialize VADER sentiment analyzer

sid = SentimentIntensityAnalyzer()

In [13]:
# Function to calculate VADER sentiment
def get_vader_sentiment(text):
    if pd.isna(text):
        return 0  # Handle missing values by returning a neutral score
    scores = sid.polarity_scores(text)
    return scores['compound']

In [14]:
# Apply the function to calculate sentiment
tweets_dataset['score_Vader'] = tweets_dataset['cleaned_tweet_text'].apply(get_vader_sentiment)

In [15]:
tweets_dataset[['cleaned_tweet_text', 'score_Vader']].head(10)

,cleaned_tweet_text,score_Vader
0,"- Awww, that's a bummer. You shoulda got Davi...",-0.0173
1,is upset that he can't update his Facebook by ...,-0.7500
2,I dived many times for the ball. Managed to sa...,0.4939
3,my whole body feels itchy and like its on fire,-0.2500
4,"no, it's not behaving at all. i'm mad. why am ...",-0.6597
5,not the whole crew,0.0000
6,Need a hug,0.4767
7,"hey long time no see! Yes.. Rains a bit ,only...",0.7450
8,nope they didn't have it,0.0000
9,que me muera ?,0.0000


In [16]:
tweets_dataset[['cleaned_tweet_text', 'score_TextBlob', 'score_Vader']].head(10)

,cleaned_tweet_text,score_TextBlob,score_Vader
0,"- Awww, that's a bummer. You shoulda got Davi...",0.216667,-0.0173
1,is upset that he can't update his Facebook by ...,0.000000,-0.7500
2,I dived many times for the ball. Managed to sa...,0.500000,0.4939
3,my whole body feels itchy and like its on fire,0.200000,-0.2500
4,"no, it's not behaving at all. i'm mad. why am ...",-0.625000,-0.6597
5,not the whole crew,0.200000,0.0000
6,Need a hug,0.000000,0.4767
7,"hey long time no see! Yes.. Rains a bit ,only...",0.270833,0.7450
8,nope they didn't have it,0.000000,0.0000
9,que me muera ?,0.000000,0.0000


In [17]:
sid = SentimentIntensityAnalyzer()

In [18]:
def sentiment_label(score):
    if score > 0:
        return 'Positive'
    elif score < 0:
        return 'Negative'
    else:
        return 'Neutral'

# Apply VADER to each tweet and create a new column for sentiment scores
tweets_dataset['compound'] = tweets_dataset['cleaned_tweet_text'].apply(lambda x: sid.polarity_scores(x)['compound'])


In [19]:

tweets_dataset['sentiment'] = tweets_dataset['compound'].apply(sentiment_label)

In [20]:
# Display the DataFrame with the sentiment labels
print(tweets_dataset[['cleaned_tweet_text', 'sentiment']])

                                       cleaned_tweet_text sentiment
0       - Awww, that's a bummer.  You shoulda got Davi...  Negative
1       is upset that he can't update his Facebook by ...  Negative
2       I dived many times for the ball. Managed to sa...  Positive
3          my whole body feels itchy and like its on fire  Negative
4       no, it's not behaving at all. i'm mad. why am ...  Negative
...                                                   ...       ...
638474  Apparetly they just started filming so imma go...   Neutral
638475  I would just like to say thank you from all of...  Positive
638476                    thanx love ! I definitely agree  Positive
638477  the ones from yesterday w/ the animals, VERY CUTE  Positive
638478  hey Andrea, how are you? This is my new twitte...  Positive

[638479 rows x 2 columns]


In [21]:

tweets_dataset[['cleaned_tweet_text', 'score_TextBlob', 'score_Vader', 'sentiment']].head(31)

,cleaned_tweet_text,score_TextBlob,score_Vader,sentiment
0,"- Awww, that's a bummer. You shoulda got Davi...",0.216667,-0.0173,Negative
1,is upset that he can't update his Facebook by ...,0.000000,-0.7500,Negative
2,I dived many times for the ball. Managed to sa...,0.500000,0.4939,Positive
3,my whole body feels itchy and like its on fire,0.200000,-0.2500,Negative
4,"no, it's not behaving at all. i'm mad. why am ...",-0.625000,-0.6597,Negative
5,not the whole crew,0.200000,0.0000,Neutral
6,Need a hug,0.000000,0.4767,Positive
7,"hey long time no see! Yes.. Rains a bit ,only...",0.270833,0.7450,Positive
8,nope they didn't have it,0.000000,0.0000,Neutral
9,que me muera ?,0.000000,0.0000,Neutral


In [22]:
tweets_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638479 entries, 0 to 638478
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tweet_ID            638479 non-null  int64  
 1   date_timestamp      638479 non-null  object 
 2   twitter_user        638479 non-null  object 
 3   cleaned_tweet_text  638479 non-null  object 
 4   score_TextBlob      638479 non-null  float64
 5   score_Vader         638479 non-null  float64
 6   compound            638479 non-null  float64
 7   sentiment           638479 non-null  object 
dtypes: float64(3), int64(1), object(4)
memory usage: 39.0+ MB


In [23]:
# Sentiment analysis functions
def get_vader_sentiment(text):
    if pd.isna(text):
        return 0
    return sid.polarity_scores(text)['compound']

def get_textblob_sentiment(text):
    if pd.isna(text):
        return 0
    return TextBlob(text).sentiment.polarity

def label_sentiment(score):
    if score > 0:
        return 'Positive'
    elif score < 0:
        return 'Negative'
    else:
        return 'Neutral'

# Apply sentiment analysis and label sentiments
tweets_dataset['vader_score'] = tweets_dataset['cleaned_tweet_text'].apply(get_vader_sentiment)
tweets_dataset['textblob_score'] = tweets_dataset['cleaned_tweet_text'].apply(get_textblob_sentiment)
tweets_dataset['vader_sentiment'] = tweets_dataset['vader_score'].apply(label_sentiment)
tweets_dataset['textblob_sentiment'] = tweets_dataset['textblob_score'].apply(label_sentiment)

In [24]:
tweets_dataset

,tweet_ID,date_timestamp,twitter_user,cleaned_tweet_text,score_TextBlob,score_Vader,compound,sentiment,vader_score,textblob_score,vader_sentiment,textblob_sentiment
0,1467810369,04-06-2009 22:19:45,_TheSpecialOne_,"- Awww, that's a bummer. You shoulda got Davi...",0.216667,-0.0173,-0.0173,Negative,-0.0173,0.216667,Negative,Positive
1,1467810672,04-06-2009 22:19:49,scotthamilton,is upset that he can't update his Facebook by ...,0.000000,-0.7500,-0.7500,Negative,-0.7500,0.000000,Negative,Neutral
2,1467810917,04-06-2009 22:19:53,mattycus,I dived many times for the ball. Managed to sa...,0.500000,0.4939,0.4939,Positive,0.4939,0.500000,Positive,Positive
3,1467811184,04-06-2009 22:19:57,ElleCTF,my whole body feels itchy and like its on fire,0.200000,-0.2500,-0.2500,Negative,-0.2500,0.200000,Negative,Positive
4,1467811193,04-06-2009 22:19:57,Karoli,"no, it's not behaving at all. i'm mad. why am ...",-0.625000,-0.6597,-0.6597,Negative,-0.6597,-0.625000,Negative,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...
638474,2072531933,06-07-2009 21:12:35,Neo_drone,Apparetly they just started filming so imma go...,0.000000,0.0000,0.0000,Neutral,0.0000,0.000000,Neutral,Neutral
638475,2072531954,06-07-2009 21:12:35,edenhig,I would just like to say thank you from all of...,0.000000,0.6124,0.6124,Positive,0.6124,0.000000,Positive,Neutral
638476,2072532046,06-07-2009 21:12:36,Gotjuicygirl,thanx love ! I definitely agree,0.312500,0.8655,0.8655,Positive,0.8655,0.312500,Positive,Positive
638477,2072532066,06-07-2009 21:12:36,whitet,"the ones from yesterday w/ the animals, VERY CUTE",0.650000,0.6965,0.6965,Positive,0.6965,0.650000,Positive,Positive


In [25]:
tweets_dataset[['cleaned_tweet_text', 'score_TextBlob', 'score_Vader', 'vader_sentiment', 'textblob_sentiment']]

,cleaned_tweet_text,score_TextBlob,score_Vader,vader_sentiment,textblob_sentiment
0,"- Awww, that's a bummer. You shoulda got Davi...",0.216667,-0.0173,Negative,Positive
1,is upset that he can't update his Facebook by ...,0.000000,-0.7500,Negative,Neutral
2,I dived many times for the ball. Managed to sa...,0.500000,0.4939,Positive,Positive
3,my whole body feels itchy and like its on fire,0.200000,-0.2500,Negative,Positive
4,"no, it's not behaving at all. i'm mad. why am ...",-0.625000,-0.6597,Negative,Negative
...,...,...,...,...,...
638474,Apparetly they just started filming so imma go...,0.000000,0.0000,Neutral,Neutral
638475,I would just like to say thank you from all of...,0.000000,0.6124,Positive,Neutral
638476,thanx love ! I definitely agree,0.312500,0.8655,Positive,Positive
638477,"the ones from yesterday w/ the animals, VERY CUTE",0.650000,0.6965,Positive,Positive


In [26]:
# Drop the column called 'Index'

tweets_dataset = tweets_dataset.drop(columns=['score_TextBlob', 'score_Vader', 'compound', 'sentiment'])

In [27]:
tweets_dataset

,tweet_ID,date_timestamp,twitter_user,cleaned_tweet_text,vader_score,textblob_score,vader_sentiment,textblob_sentiment
0,1467810369,04-06-2009 22:19:45,_TheSpecialOne_,"- Awww, that's a bummer. You shoulda got Davi...",-0.0173,0.216667,Negative,Positive
1,1467810672,04-06-2009 22:19:49,scotthamilton,is upset that he can't update his Facebook by ...,-0.7500,0.000000,Negative,Neutral
2,1467810917,04-06-2009 22:19:53,mattycus,I dived many times for the ball. Managed to sa...,0.4939,0.500000,Positive,Positive
3,1467811184,04-06-2009 22:19:57,ElleCTF,my whole body feels itchy and like its on fire,-0.2500,0.200000,Negative,Positive
4,1467811193,04-06-2009 22:19:57,Karoli,"no, it's not behaving at all. i'm mad. why am ...",-0.6597,-0.625000,Negative,Negative
...,...,...,...,...,...,...,...,...
638474,2072531933,06-07-2009 21:12:35,Neo_drone,Apparetly they just started filming so imma go...,0.0000,0.000000,Neutral,Neutral
638475,2072531954,06-07-2009 21:12:35,edenhig,I would just like to say thank you from all of...,0.6124,0.000000,Positive,Neutral
638476,2072532046,06-07-2009 21:12:36,Gotjuicygirl,thanx love ! I definitely agree,0.8655,0.312500,Positive,Positive
638477,2072532066,06-07-2009 21:12:36,whitet,"the ones from yesterday w/ the animals, VERY CUTE",0.6965,0.650000,Positive,Positive


In [28]:
tweets_dataset.to_csv('sentiment_score_and_labeled_tweets.csv', index=False)